# 安裝套件

In [ ]:
# 套件安裝指令
!pip install --extra-index-url https://download.pytorch.org/whl/cu125 torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 accelerate==1.8.1 transformers datasets evaluate scikit-learn google.colab

# 掛載 Google Drive

In [ ]:
# 會需要使用者同意授權/存取 Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 切換目錄 (Colab 預設目錄為 /content，使用 %cd 切換目錄)
%cd /content/drive/MyDrive/

# 觀看系統設定

In [ ]:
!lsb_release -a

In [ ]:
!nvidia-smi

In [ ]:
!nvcc -V

In [ ]:
!python -V

# 微調模型

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))

# 匯入套件

In [ ]:
'''
AutoTokenizer：這有助於將我們的文字資料標記為 BERT 可以理解的格式。 「Auto」前綴意味著它可以為各種模型推斷適當的分詞器。
AutoModelForSequenceClassification：一個通用的類別，是用於「序列分類」任務的模型架構。「Auto」前綴使其在各種預訓練模型中具有通用性。
TrainingArguments：定義訓練配置的設定，例如 learning rateb、batch size 和 epoch。
Trainer：用於訓練和評估，使 finetune 變得簡單。
pipeline：使用模型的模型。
DataCollat​​eWithPadding：確保我們分詞化後的資料，以一致的長度串接在一起，並在必要時增加 padding。這對於訓練的穩定性和效率至關重要。
'''
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

import random
from sklearn.metrics import f1_score

# 基本參數設定

In [ ]:
model_name = 'google-bert/bert-base-chinese' # 預訓練模型名稱
max_seq_length = 512 # 可訓練的序列最大長度
num_labels = 2 # 二元分類
output_dir = './output' # 輸出模型資料夾

# 載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 讀取 .txt 文件

In [ ]:
# 設定隨機種子，確保實驗可重現
seed=42

# 設定資料路徑
file_path = './reviews.txt'

# 讀取資料
with open(file_path, "r", encoding='utf-8') as file:
    # 將每一行資料以 list 型態回傳
    lines = file.readlines()

    # 洗牌 (記得設定 random seed，確保每次洗牌結果一樣)
    random.seed(seed)
    random.shuffle(lines)

    # 整合訓練資料
    sentences = []
    labels = []

    # 逐行讀取資料
    for line in lines:
        # 每一行資料的 tab (\t) 作為分隔符號
        parts = line.strip().split('\t')

        # 確保每一行資料都有兩個部分
        if len(parts) == 2:
            sentences.append(parts[0])
            labels.append(int(parts[1]))
        else:
            print(f'格式錯誤的行號: {line}')

# 轉換成 huggingface trainer 可以使用的 datasets

In [ ]:
# 建立 Dataset
dataset = Dataset.from_dict({
    'sentences': sentences,
    'labels': labels
})

# 回傳切分資料 (訓練 和 驗證)
dataset = dataset.train_test_split(test_size=0.2)
'''
print(dataset) 的內容如下:

DatasetDict({
    train: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 6212
    })
    test: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 1554
    })
})
'''

# 預處理資料
def preprocess_data(dataset):
    # 將句子轉換為 token (tokenization)
    return tokenizer(
        dataset['sentences'], 
        truncation=True, 
        padding=True, 
        return_tensors='pt', 
        max_length=max_seq_length
    )

# 轉換資料
train_data = dataset['train'].map(preprocess_data, batched=True)
valid_data = dataset['test'].map(preprocess_data, batched=True)

# 建立 DatasetDict
dataset_dict = DatasetDict({
    'train': train_data,
    'test': valid_data
})

# 設定模型評估指標

In [ ]:
# 計算模型評估指標
def compute_metrics(predicted_results):
    # 取得真實標籤
    labels = predicted_results.label_ids

    # 取得預測結果
    preds = predicted_results.predictions.argmax(-1)
    '''
    為什麼是 argmax(-1)？

    import numpy as np
    predictions = np.array([
        [0.1, 0.9],
        [0.8, 0.2],
        [0.4, 0.6],
    ])
    preds = predictions.argmax(axis=-1)
    print(preds) 

    np.argmax() 會找出最大的值，並回傳索引值
    - 如果是 axis=0，則會是 [1, 0] (0.8 和 0.9)，因為由上而下，對每一個欄位取最大值
    - 如果是 axis=1，則會是 [1, 0, 1] (0.9、0.8 和 0.6)，因為由左而右，對每一個列取最大值
    - 如果是 axis=-1，則會以最後一個維度為基礎 (上面範例的維度是 2)，取最大值，所以 axis=-1 等同於 axis=1，結果也是 [1, 0, 1]
    '''

    # 計算 F1 score (二元分類選 binary，多元分類可選 micro, macro, weighted)
    # 可參考: https://blog.csdn.net/qq_40671063/article/details/130447922
    f1 = f1_score(labels, preds, average='binary') 
    return {
        'f1': f1,
    }

# 微調模型

In [ ]:
# 讀取模型
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=num_labels
)

# 設定訓練參數
training_args = TrainingArguments(
    output_dir=output_dir, # 輸出資料夾
    overwrite_output_dir=True,
    num_train_epochs=3, # 訓練回合數
    per_device_train_batch_size=32, # 批次大小
    per_device_eval_batch_size=32, # 批次大小
    gradient_accumulation_steps=2, # 梯度累積步數，主要是為了讓較小的 GPU 也能訓練較大的 batch size
    learning_rate=0.00005 * 2, # 預設會將每個小 batch 的 loss 除以累積步數（loss / 2），讓最終梯度等同於單一大 batch，但這會讓梯度變小（等於將 learning rate 除以 2），所以要手動將 learning rate 乘以累積步數來抵消
    warmup_steps=50, # 預熱步數，主要是讓 learning rate 從 0 緩慢增加到設定的 learning rate，避免一開始學習率過大導致模型不穩定
    weight_decay=0.01, # 權重衰減，主要是為了正則化，避免過擬合，讓 loss 曲線較平滑
    eval_strategy="steps", # epoch, steps, no
    eval_steps=70, # 每隔多少步驟評估一次
    save_strategy="steps", # epoch, steps, no
    save_steps=70, # 每隔多少步驟儲存一次
    save_total_limit=2, # 最多儲存模型數量
    load_best_model_at_end=True, # Trainer 會追蹤 metric_for_best_model（預設 eval_loss，越小越好）來判斷最佳模型，訓練結束時自動將該最佳檢查點載入記憶體覆蓋最終模型
    seed=seed, # 隨機種子，主要是為了確保實驗可重現
    lr_scheduler_type="linear", # https://blog.csdn.net/muyao987/article/details/139319466
    report_to="none", # 關閉內建的實驗追蹤功能 (如 wandb, tensorboard)
)

# 設定 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['test'],
    data_collator=None, # DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# 開始訓練
trainer.train()

# 儲存模型
trainer.save_model(output_dir)

# 儲存 tokenizer
tokenizer.save_pretrained(output_dir)

# 拿微調好的模型，進行預測

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
)
from pprint import pprint

# 先前微調完 model 與 tokenizer 的輸出資料夾
output_dir = './output'

# 載入模型與 tokenizer
model = AutoModelForSequenceClassification.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# 建立分類管線
pipe = pipeline(task='text-classification', model=model, tokenizer=tokenizer, device=0)

# 測試分類管線
list_text = [
    '這個房間真的不錯，服務人員也很親切，下次還會再來！',
    '這個房間真的很爛，服務人員也很差，下次不會再來！',
    '一般般',
]

# 執行分類
result = pipe(list_text)

# 輸出結果
pprint(result)